In [1]:
#ライブラリをインポート
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#欠損値補完用のimputeをインポート
from tsfresh.utilities.dataframe_functions import impute

In [2]:
#解析データをdfとして取り込み、id=1の列を追加（id=1の行がtsfreshの解析対象になるので、全行にid=1を割り当てる）
df = pd.read_excel("入力データ.xlsx", sheet_name="波形データ", index_col=0)
nug = pd.read_excel("入力データ.xlsx", sheet_name="Nugget", index_col=0)


#データの先頭5行を確認（データにミスが無いか確認）
df.head()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
No,,,,,,,,,,,,,,,,,,,,,
time_1,0.990607,1.668915,1.776132,1.749081,1.848263,1.770670,1.970585,1.918033,1.866176,1.943025,...,1.986662,1.951425,1.935142,1.955834,1.989123,1.946936,1.984508,1.888336,1.845127,1.985932
time_2,0.841471,1.327137,1.734427,0.993954,1.440724,1.081505,1.945080,1.695870,1.103119,1.503455,...,1.261544,1.411306,1.466108,1.767686,1.534756,1.723808,1.838034,1.826516,0.968491,1.726035
time_3,0.909297,1.111378,1.230344,1.444847,1.702474,1.249707,1.676757,1.905421,1.810481,1.751209,...,1.968013,1.680663,1.910978,1.355728,1.004164,1.591883,1.745091,1.539681,1.694292,1.023086
time_4,0.141120,0.607504,0.762426,1.019978,1.249574,0.875411,1.514007,1.622256,1.866176,1.716741,...,1.642331,1.458652,1.711695,1.123713,1.520672,1.502337,1.618326,1.836457,1.454067,1.800716
time_5,-0.756802,0.002907,-0.610336,0.253292,0.351292,0.303609,0.748752,1.128678,0.859294,1.445477,...,1.564976,1.574032,1.413235,1.124842,1.253174,1.189655,1.182477,1.888336,1.554944,1.117506


In [3]:
df_3 = pd.DataFrame(index=range(len(df.columns)+1), columns=[''])

x=0
le=1

while x < len(df):
    while (x+le) <=  len(df):
        df_2 = df.iloc[x:x+le+1, :]
        
        df_3[df.index[x] + "-" + df.index[x+le] + "_最大値（生値）"] = df_2.max()
        df_3[df.index[x] + "-" + df.index[x+le] + "_最小値（生値）"] = df_2.min()
        df_3[df.index[x] + "-" + df.index[x+le] + "_平均値"] = df_2.mean()
        df_3[df.index[x] + "-" + df.index[x+le] + "_変化量"] = df_2.max() - df_2.min()
        df_3[df.index[x] + "-" + df.index[x+le] + "_積分（生値）"] = df_2.abs().sum()
        df_3[df.index[x] + "-" + df.index[x+le] + "_標準偏差"] = df_2.std()
        
        df_3[df.index[x] + "-" + df.index[x+le] + "_最大値（オフセット）"] = (df_2 - df_2.iloc[0:1,:].values).max()
        df_3[df.index[x] + "-" + df.index[x+le] + "_最小値（オフセット）"] = (df_2 - df_2.iloc[0:1,:].values).min()
        df_3[df.index[x] + "-" + df.index[x+le] + "_積分（オフセット）"] = (df_2 - df_2.iloc[0:1,:].values).abs().sum()
        
        le += 1
    
    x += 1
    le = 1


df_3 = df_3.drop(df_3.columns[0], axis=1)
df_3 = df_3.drop(df_3.index[0], axis=0)
df_3 = impute(df_3)

C:\Users\yuuuu\anaconda3\lib\site-packages\tsfresh\utilities\dataframe_functions.py:171: RuntimeWarning: The columns ['29-30_標準偏差'] did not have any finite values. Filling with zeros.
  warnings.warn("The columns {} did not have any finite values. Filling with zeros.".format(


In [4]:
from sklearn.feature_selection import SelectKBest,f_regression

x = df_3
y = nug.transpose()

In [5]:
selector = SelectKBest(score_func=f_regression, k=30) 
selector.fit(x, y)
mask = selector.get_support()

C:\Users\yuuuu\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\yuuuu\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:302: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms


In [6]:
x = x.transpose()
x.insert(0,"選別フラグ", mask)

In [7]:
x = x.transpose()
x.insert(0,"ナゲット径", y)
x = x.transpose()

In [8]:
with pd.ExcelWriter('出力結果.xlsx') as writer:
    x.to_excel(writer, sheet_name='出力結果', encoding="cp932")